Version 1.0.1

# Check your versions

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm 

for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

numpy 1.13.3
pandas 0.20.3
scipy 0.19.1
sklearn 0.19.1
lightgbm 2.0.6


**Important!** There is a huge chance that the assignment will be impossible to pass if the versions of `lighgbm` and `scikit-learn` are wrong. The versions being tested:

    numpy 1.13.1
    pandas 0.20.3
    scipy 0.19.1
    sklearn 0.19.0
    ligthgbm 2.0.6
    

To install an older version of `lighgbm` you may use the following command:
```
pip uninstall lightgbm
pip install lightgbm==2.0.6
```

# Ensembling

In this programming assignment you are asked to implement two ensembling schemes: simple linear mix and stacking.

We will spend several cells to load data and create feature matrix, you can scroll down this part or try to understand what's happening.

In [2]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


from tqdm import tqdm_notebook

from itertools import product


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

# Load data subset

Let's load the data from the hard drive first.

In [3]:
sales = pd.read_csv('../Final Project/sales_train.csv.gz')
shops = pd.read_csv('../Final Project/shops.csv')
items = pd.read_csv('../Final Project/items.csv')
item_cats = pd.read_csv('../Final Project/item_categories.csv')
ww = pd.read_csv('../Final Project/test.csv.gz')


In [4]:
print (sales.head(5))
print (ww.head(5))
del sales['date'] 
del sales['item_price']
del ww['ID']
ww['item_cnt_day'] = 0
ww['date_block_num'] = 34

print (sales.head(5))
print (ww.head(5))

sales = pd.concat([sales,ww],ignore_index=True)


         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
   date_block_num  shop_id  item_id  item_cnt_day
0               0       59    22154           1.0
1               0       25     2552           1.0
2               0       25     2552          -1.0
3               0       25     2554           1.0
4               0       25     2555           1.0
   shop_id  item_id  item_cnt_day  date_block_num
0        5     5037             0              34
1        5  

In [5]:
print (sales.head(10))
print (sales.tail(10))

   date_block_num  item_cnt_day  item_id  shop_id
0               0           1.0    22154       59
1               0           1.0     2552       25
2               0          -1.0     2552       25
3               0           1.0     2554       25
4               0           1.0     2555       25
5               0           1.0     2564       25
6               0           1.0     2565       25
7               0           1.0     2572       25
8               0           1.0     2572       25
9               0           3.0     2573       25
         date_block_num  item_cnt_day  item_id  shop_id
3150039              34           0.0     3280       45
3150040              34           0.0     4393       45
3150041              34           0.0     4352       45
3150042              34           0.0    18049       45
3150043              34           0.0    18027       45
3150044              34           0.0    18454       45
3150045              34           0.0    16188       45
31

# Get a feature matrix

We now need to prepare the features. This part is all implemented for you.

In [6]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
all_data = downcast_dtypes(all_data)
del grid, gb 
gc.collect();

/home/danielchan/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [7]:
print (all_data.shape)

(11128050, 6)


After creating a grid, we can calculate some features. We will use lags from [1, 2, 3, 4, 5, 12] months ago.

In [8]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in tqdm_notebook(shift_range):
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

A Jupyter Widget

To this end, we've created a feature matrix. It is stored in `all_data` variable. Take a look:

In [9]:
all_data.head(5)

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,54,10297,12,4.0,8198.0,23.0,3.0,42.0,10055.0,0.0,2.0,7978.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
1,54,10296,12,3.0,8198.0,17.0,0.0,24.0,10055.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,38
2,54,10298,12,14.0,8198.0,182.0,21.0,369.0,10055.0,119.0,1309.0,7978.0,7.0,144.0,6676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
3,54,10300,12,3.0,8198.0,26.0,1.0,54.0,10055.0,31.0,361.0,7978.0,0.0,53.0,6676.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
4,54,10284,12,1.0,8198.0,3.0,0.0,4.0,10055.0,0.0,3.0,7978.0,0.0,5.0,6676.0,0.0,3.0,7827.0,0.0,10.0,7792.0,0.0,0.0,0.0,57


# Train/test split

For a sake of the programming assignment, let's artificially split the data into train and test. We will treat last month data as the test set.

In [10]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

sub_block = dates.max()
last_block = dates.max() - 1
print('Test `date_block_num` is %d' % last_block)
print('Submit `date_block_num` is %d' % sub_block)

Test `date_block_num` is 33
Submit `date_block_num` is 34


In [11]:
print (dates.tail(10))

6639284    34
6639285    34
6639286    34
6639287    34
6639288    34
6639289    34
6639290    34
6639291    34
6639292    34
6639293    34
Name: date_block_num, dtype: int32


In [49]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)
X_sub  =  all_data.loc[dates == sub_block].drop(to_drop_cols, axis=1)


y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

In [50]:
print (X_train.shape)

print (X_test.shape)

print (X_sub.shape)

(6186922, 21)
(238172, 21)
(214200, 21)


# First level models 

You need to implement a basic stacking scheme. We have a time component here, so we will use ***scheme f)*** from the reading material. Recall, that we always use first level models to build two datasets: test meta-features and 2-nd level train-metafetures. Let's see how we get test meta-features first. 

### Test meta-features

Firts, we will run *linear regression* on numeric columns and get predictions for the last month.

And the we run *LightGBM*.

In [40]:
# Linear Regression

lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_test.values)

pred_lr = pred_lr.clip(0,20.)
y_test = y_test.clip(0,20.)

print('Test RMSE for linreg is %f' % mean_squared_error(y_test, pred_lr))

Test RMSE for linreg is 1.008470


In [41]:
# Lasso Regression
from sklearn.linear_model import Lasso

lassoReg = Lasso(alpha=0.72)
lassoReg.fit(X_train.values, y_train) 
pred_lasso = lassoReg.predict(X_test.values)

pred_lasso = pred_lasso.clip(0,20.)
y_test = y_test.clip(0,20.)

print('Test RMSE for lassoReg is %f' % mean_squared_error(y_test, pred_lasso))

Test RMSE for lassoReg is 0.991579


In [42]:
from sklearn.linear_model import Ridge

ridgeReg = Ridge(alpha=0.5,normalize=True )

print (ridgeReg)
ridgeReg.fit(X_train.values, y_train) 
pred_ridge = ridgeReg.predict(X_test.values)

pred_ridge = pred_ridge.clip(0,20.)
y_test = y_test.clip(0,20.)

print('Test RMSE for ridgeReg is %f' % mean_squared_error(y_test, pred_ridge))

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001)
Test RMSE for ridgeReg is 0.986719


In [43]:
lgb_params = {
               'feature_fraction': 0.5,
               'metric': 'rmse',
               'nthread':8, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.01, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb = model.predict(X_test)

pred_lgb = pred_lgb.clip(0,20.)
y_test = y_test.clip(0,20.)

print('Test RMSE for LightGBM is %f' % mean_squared_error(y_test, pred_lgb))

Test RMSE for LightGBM is 0.905192


Finally, concatenate test predictions to get test meta-features.

In [52]:
#X_test_level2 = np.c_[pred_lr, pred_lgb] 
X_test_level2 = np.c_[pred_lasso, pred_lgb] 


print (X_test_level2.shape)

(238172, 2)


# Submit Meta-Features

In [51]:
print (X_sub.shape)

#sub_lr = lr.predict(X_sub.values).clip(0,20.)
sub_lr = lassoReg.predict(X_sub.values).clip(0,20.)
sub_lgb = model.predict(X_sub).clip(0,20.)

X_sub_level2 = np.c_[sub_lr, sub_lgb] 
print (X_sub_level2.shape)

(214200, 21)
(214200, 2)


### Train meta-features

**Now it is your turn to write the code**. You need to implement ***scheme f)*** from the reading material. Here, we will use duration **T** equal to month and **M=15**.  

That is, you need to get predictions (meta-features) from *linear regression* and *LightGBM* for months 27, 28, 29, 30, 31, 32. Use the same parameters as in above models.

In [53]:
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

y_train_level2 = y_train_level2.clip(0,20.)

In [54]:
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])


# Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''      
    
    #  YOUR CODE GOES HERE

    Xtemp = X_train[dates_train < cur_block_num]
    ytemp = y_train[dates_train < cur_block_num]
    
    Xtemp_1 = X_train[dates_train == cur_block_num]
    
    lassoReg.fit(Xtemp.values, ytemp)
    pred_lr = lassoReg.predict(Xtemp_1.values)
    
    model = lgb.train(lgb_params, lgb.Dataset(Xtemp, label=ytemp), 100)
    pred_lgb = model.predict(Xtemp_1)

    #X_train_level2[dates_train_level2==cur_block_num,:2] = np.c_[pred_lr, pred_lgb]
    X_train_level2[dates_train_level2==cur_block_num,:] = np.c_[pred_lr, pred_lgb]

print (X_train_level2.shape)
print(X_train_level2.mean(axis=0))    
# Sanity check
#assert np.all(np.isclose(X_train_level2.mean(axis=0), [ 1.50148988,  1.38811989]))

27
28
29
30
31
32
(1376192, 2)
[ 0.30705778  0.30374423]


Remember, the ensembles work best, when first level models are diverse. We can qualitatively analyze the diversity by examinig *scatter plot* between the two metafeatures. Plot the scatter plot below. 

In [55]:
print (dates_train_level2.shape)
print (X_train_level2.shape)

(1376192,)
(1376192, 2)


# Ensembling

Now, when the meta-features are created, we can ensemble our first level models.

### Simple convex mix

Let's start with simple linear convex mix:

$$
mix= \alpha\cdot\text{linreg_prediction}+(1-\alpha)\cdot\text{lgb_prediction}
$$

We need to find an optimal $\alpha$. And it is very easy, as it is feasible to do grid search. Next, find the optimal $\alpha$ out of `alphas_to_try` array. Remember, that you need to use train meta-features (not test) when searching for $\alpha$. 

In [56]:
alphas_to_try = np.linspace(0, 1, 1001)

# YOUR CODE GOES HERE
r2_train_simple_mix = 2.0

for alpha in alphas_to_try:
    mix = alpha*X_train_level2[:,0] + (1.0-alpha)*X_train_level2[:,1]   
    mix = mix.clip(0,20.)
    #print (np.amax(y_train_level2 - mix))

    rtemp = mean_squared_error(y_train_level2, mix)
    if (rtemp < r2_train_simple_mix) :
        r2_train_simple_mix = rtemp
        best_alpha = alpha
        

print('Best alpha: %f; Corresponding RMSE score on train: %f' % (best_alpha, r2_train_simple_mix))

Best alpha: 0.281000; Corresponding RMSE score on train: 0.737175


Now use the $\alpha$ you've found to compute predictions for the test set 

In [57]:
# YOUR CODE GOES HERE
test_preds = best_alpha*X_test_level2[:,0]  +  (1.0-best_alpha)*X_test_level2[:,1]

test_preds = test_preds.clip(0,20.)
y_test = y_test.clip(0,20.)

print (X_test_level2.shape)
print (y_test.shape)
print (test_preds.shape)

r2_test_simple_mix = mean_squared_error(y_test, test_preds)     # YOUR CODE GOES HERE

print('Test RMSE for simple mix is %f' % r2_test_simple_mix)

(238172, 2)
(238172,)
(238172,)
Test RMSE for simple mix is 0.892550


### Stacking

Now, we will try a more advanced ensembling technique. Fit a linear regression model to the meta-features. Use the same parameters as in the model above.

In [58]:

linReg = LinearRegression(normalize=False)
linReg.fit(X_train_level2, y_train_level2)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Compute RSME on the train and test sets.

In [59]:
train_preds = linReg.predict(X_train_level2)  
train_preds = train_preds.clip(0,20.)                                       # YOUR CODE GOES HERE
r2_train_stacking = mean_squared_error(y_train_level2, train_preds)         # YOUR CODE GOES HERE

test_preds = linReg.predict(X_test_level2)               # YOUR CODE GOES HERE
test_preds = test_preds.clip(0,20.)                                       # YOUR CODE GOES HERE

r2_test_stacking = mean_squared_error(y_test, test_preds)   # YOUR CODE GOES HERE

print('Train RMSE for stacking is %f' % r2_train_stacking)
print('Test  RMSE for stacking is %f' % r2_test_stacking)



Train RMSE for stacking is 0.911484
Test  RMSE for stacking is 1.082497


Interesting, that the score turned out to be lower than in previous method. Although the model is very simple (just 3 parameters) and, in fact, mixes predictions linearly, it looks like it managed to overfit. **Examine and compare** train and test scores for the two methods. 

And of course this particular case does not mean simple mix is always better than stacking.

We all done! Submit everything we need to the grader now.

In [60]:
# YOUR CODE GOES HERE
sub_preds = best_alpha*X_sub_level2[:,0]  +  (1.0-best_alpha)*X_sub_level2[:,1]
sub_preds = sub_preds.clip(0,20.)

print (X_sub.shape)
print (sub_preds.shape)

X_sub['target'] = sub_preds

ww = pd.read_csv('../Final Project/test.csv.gz')


submit = pd.merge(ww,X_sub,how='left',on=['shop_id', 'item_id']).fillna(0)

print (ww.shape)
print (submit.shape)

#print (submit.head(10))
#print (submit[submit.date_block_num == 0.0].count())

submit.to_csv(path_or_buf='submit_lasso.csv',columns=['ID','target'], header=['ID', 'item_cnt_month'], index=False)

(214200, 21)
(214200,)
(214200, 3)
(214200, 23)


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=10,n_jobs=8)

print (neigh)
print ('Fitting')
neigh.fit(X_train.values, y_train) 
print ('Predicting')

pred_knn = neigh.predict(X_test.values)

pred_knn = pred_knn.clip(0,20.)
y_test = y_test.clip(0,20.)

print('Test RMSE for knn is %f' % mean_squared_error(y_test, pred_knn))



In [ ]:
# SVM Regression
from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_lin = SVR(kernel='linear', C=1e3)

#pred_rbf = svr_rbf.fit(X_train.values, y_train).predict(X_test.values)
pred_lin = svr_lin.fit(X_train.values, y_train).predict(X_test.values)

#pred_rbf = pred_rbf.clip(0,20.)
pred_lin = pred_lin.clip(0,20.)

y_test = y_test.clip(0,20.)

print('Test RMSE for SVR Linear is %f' % mean_squared_error(y_test, pred_lin))
#print('Test RMSE for SVR Rbf is %f' % mean_squared_error(y_test, pred_rbf))